In [191]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [192]:
cd "/content/drive/My Drive/Colab Notebooks/Predicta"

/content/drive/My Drive/Colab Notebooks/Predicta


In [193]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder  # type: ignore
from sklearn.model_selection import train_test_split  # type: ignore
from sklearn.metrics import accuracy_score  # type: ignore
from xgboost import XGBClassifier  # type: ignore
from sklearn.model_selection import train_test_split


In [194]:
df = pd.read_csv("filtered_data_with_classes.csv")
df.head()

,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise_num,sunset_num
0,D0003,C001,-0.3,0,-0.526829,-0.946667,-0.285714,45.0,0.772727,0.333333,-0.365019,0.0,0.0,-0.185430,0,-0.272727,0.483221
1,D0004,C001,-0.6,1,-0.282927,-0.013333,0.714286,0.0,0.227273,-0.666667,-0.593156,0.0,0.0,-0.278146,0,-0.250000,0.456376
2,D0007,C001,-0.2,2,-0.487805,1.053333,0.285714,0.0,0.772727,0.000000,-0.288973,0.0,0.0,-0.019868,1,-0.204545,0.389262
3,D0019,C001,-0.4,1,-0.526829,-0.586667,0.571429,0.0,0.227273,-0.666667,-0.441065,0.0,0.0,-0.470199,2,0.068182,0.120805
4,D0028,C002,-0.4,2,-0.526829,-0.460000,-0.428571,0.0,-0.454545,-0.333333,-0.441065,0.0,0.0,-0.470199,0,0.000000,0.510067


In [195]:
# Encoding categorical variables (if 'condition_text' is categorical)
label_encoder = LabelEncoder()
df['condition_text'] = label_encoder.fit_transform(df['condition_text'])
df['city_id'] = label_encoder.fit_transform(df['city_id'])

# Separating features and target variable
X = df.drop(['condition_text', "day_id"], axis=1)  # Features
y = df['condition_text']  # Target

# Step 2: Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

In [196]:
model = XGBClassifier(
    eval_metric='mlogloss',
    learning_rate=0.01,
    max_depth=5,
    n_estimators=200,
    subsample=0.9,
    gamma=0,
    min_child_weight=1,
    reg_alpha=0.09,
    reg_lambda=0.85,
    colsample_bytree=0.8,
)
print(model.get_xgb_params())

model.fit(X, y)

{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.8, 'device': None, 'eval_metric': 'mlogloss', 'gamma': 0, 'grow_policy': None, 'interaction_constraints': None, 'learning_rate': 0.01, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 5, 'max_leaves': None, 'min_child_weight': 1, 'monotone_constraints': None, 'multi_strategy': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': 0.09, 'reg_lambda': 0.85, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.9, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
from sklearn.model_selection import GridSearchCV

model = XGBClassifier(
    eval_metric='mlogloss',
    learning_rate=0.01,
    max_depth=5,
    n_estimators=200,
    subsample=0.9,
    gamma=0,
    min_child_weight=1,
    colsample_bytree=0.8,
    reg_alpha=0.2,
)
# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],  # Fraction of features used per tree
    #Minimum sum of instance weight (hessian) needed in a child
    'min_child_weight': [1, 2, 3],
    # Minimum loss reduction required to make a further partition on a leaf node
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0.08, 0.09, 0.15, 0.2, 0.25],  # L1 regularization term on weights
    'reg_lambda': [0.9, 0.85, 0.8, 0.75]  # L2 regularization term on weights
}

# Instantiate the GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X, y)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy found: ", grid_search.best_score_)

Fitting 3 folds for each of 43740 candidates, totalling 131220 fits


In [173]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9375


In [174]:
df_input = pd.read_csv('filtered_input.csv')
df_input.head()

,day_id,city_id,temperature_celsius,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise_num,sunset_num
0,D0001,C001,0.5,-0.1375,0.355030,-1.000,0.0,-1.000000,1.0,0.271845,0.0,5.0,-0.218045,1,-0.309524,0.541667
1,D0002,C001,0.0,-0.1375,0.118343,-1.000,0.0,-0.208333,1.0,0.000000,0.0,0.0,0.646617,0,-0.285714,0.527778
2,D0005,C001,-0.4,-0.4500,-0.343195,0.625,0.0,0.666667,-0.5,-0.504854,0.0,0.0,-0.436090,0,-0.238095,0.486111
3,D0006,C001,-0.2,-0.4500,-0.319527,0.625,0.0,0.416667,-0.5,-0.349515,0.0,0.0,-0.270677,0,-0.238095,0.458333
4,D0008,C001,-0.1,1.6250,1.065089,-0.375,0.0,-1.041667,1.0,-0.271845,0.0,0.0,0.187970,0,-0.190476,0.416667


In [175]:
df_input_processed = df_input.drop(['day_id'], axis=1)
df_input_processed['city_id'] = label_encoder.fit_transform(df_input_processed['city_id'])

In [176]:
lower_bound = 0.6
upper_bound = 0.8
step = 0.05

# Generate confidence levels
confidence_levels = np.arange(lower_bound, upper_bound + step, step).tolist()
# confidence_levels = [95]

for confidence_threshold in confidence_levels:
    probabilities = model.predict_proba(df_input_processed)
    # print(probabilities.shape)
    # print(probabilities[:5])
    predicted_classes = np.argmax(probabilities, axis=1)
    # print(predicted_classes.shape)
    # print(predicted_classes[:5])
    # Filter records based on the current confidence threshold
    max_probabilities = np.max(probabilities, axis=1)
    # print(max_probabilities.shape)
    sure_indices = np.where(max_probabilities >= confidence_threshold)[0]
    print("sure count", sure_indices.shape)
    sure_records_df = df_input_processed.iloc[sure_indices].copy()
    sure_labels = predicted_classes[sure_indices]
    sure_records_df['predicted_labels'] = sure_labels

    # Split the new dataset into training and testing subsets
    X_sure = sure_records_df.drop('predicted_labels', axis=1)
    y_sure = sure_records_df['predicted_labels']

    # Merge with previous training data
    X_added = pd.concat([X_train, X_sure], ignore_index=True)
    y_added = pd.concat([y_train, y_sure], ignore_index=True)

    # Retrain the model on this new training subset
    model_retrained = XGBClassifier(
        eval_metric='mlogloss',
        learning_rate=0.01,
        max_depth=5,
        n_estimators=200,
        subsample=0.9,
        gamma=0,
        min_child_weight=1,
        reg_alpha=0.09,
        reg_lambda=0.85,
        colsample_bytree=0.8,
    )
    model_retrained.fit(X_added, y_added)
    y_pred = model_retrained.predict(X_test)

    # Evaluate and print accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Confidence Threshold: {confidence_threshold: .2f}, Accuracy: {accuracy:.4f}")

sure count (1114,)
Confidence Threshold:  0.60, Accuracy: 0.7917
sure count (870,)
Confidence Threshold:  0.65, Accuracy: 0.7917
sure count (674,)
Confidence Threshold:  0.70, Accuracy: 0.8125
sure count (384,)
Confidence Threshold:  0.75, Accuracy: 0.8125
sure count (0,)
Confidence Threshold:  0.80, Accuracy: 0.8125
sure count (0,)
Confidence Threshold:  0.85, Accuracy: 0.8125


In [190]:
confidence_threshold = 0.75
probabilities = model.predict_proba(df_input_processed)
predicted_classes = np.argmax(probabilities, axis=1)
max_probabilities = np.max(probabilities, axis=1)

sure_indices = np.where(max_probabilities >= confidence_threshold)[0]
print("sure count", sure_indices.shape)
sure_records_df = df_input_processed.iloc[sure_indices].copy()
sure_labels = predicted_classes[sure_indices]
sure_records_df['predicted_labels'] = sure_labels

# Split the new dataset into training and testing subsets
X_sure = sure_records_df.drop('predicted_labels', axis=1)
y_sure = sure_records_df['predicted_labels']

# Merge with previous training data
X_added = pd.concat([X, X_sure], ignore_index=True)
y_added = pd.concat([y, y_sure], ignore_index=True)

# Retrain the model on this new training subset
model_retrained = XGBClassifier(
    eval_metric='mlogloss',
    learning_rate=0.01,
    max_depth=5,
    n_estimators=200,
    subsample=0.9,
    gamma=0,
    min_child_weight=1,
    reg_alpha=0.09,
    reg_lambda=0.85,
    colsample_bytree=0.8,
)
model_retrained.fit(X_added, y_added)
y_pred = model_retrained.predict(X_test)

# Evaluate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Confidence Threshold: {confidence_threshold: .2f}, Accuracy: {accuracy:.4f}")

sure count (384,)
Confidence Threshold:  0.75, Accuracy: 0.9167


In [180]:
confidence_threshold = 0.60
probabilities = model.predict_proba(df_input_processed)
predicted_classes = np.argmax(probabilities, axis=1)
max_probabilities = np.max(probabilities, axis=1)

sure_indices = np.where(max_probabilities >= confidence_threshold)[0]
print("sure count", sure_indices.shape)
sure_records_df = df_input_processed.iloc[sure_indices].copy()
sure_labels = predicted_classes[sure_indices]
sure_records_df['predicted_labels'] = sure_labels

# Split the new dataset into training and testing subsets
X_sure = sure_records_df.drop('predicted_labels', axis=1)
y_sure = sure_records_df['predicted_labels']

# Merge with previous training data
X_added = pd.concat([X, X_sure], ignore_index=True)
y_added = pd.concat([y, y_sure], ignore_index=True)

# Retrain the model on this new training subset
model_retrained = XGBClassifier(
    eval_metric='mlogloss',
    learning_rate=0.01,
    max_depth=5,
    n_estimators=200,
    subsample=0.9,
    gamma=0,
    min_child_weight=1,
    reg_alpha=0.09,
    reg_lambda=0.85,
    colsample_bytree=0.8,
)
model_retrained.fit(X_added, y_added)
y_pred = model_retrained.predict(X_test)

# Evaluate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Confidence Threshold: {confidence_threshold: .2f}, Accuracy: {accuracy:.4f}")

sure count (1114,)
Confidence Threshold:  0.60, Accuracy: 0.9583


In [181]:
y_pred = model_retrained.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9583


In [182]:
print(sure_records_df.count())
print(df_input_processed.count())

city_id                     1114
temperature_celsius         1114
wind_kph                    1114
wind_degree                 1114
pressure_mb                 1114
precip_mm                   1114
humidity                    1114
cloud                       1114
feels_like_celsius          1114
visibility_km               1114
uv_index                    1114
gust_kph                    1114
air_quality_us-epa-index    1114
sunrise_num                 1114
sunset_num                  1114
predicted_labels            1114
dtype: int64
city_id                     2414
temperature_celsius         2414
wind_kph                    2414
wind_degree                 2414
pressure_mb                 2414
precip_mm                   2414
humidity                    2414
cloud                       2414
feels_like_celsius          2414
visibility_km               2414
uv_index                    2414
gust_kph                    2414
air_quality_us-epa-index    2414
sunrise_num                 24

In [183]:
import json
with open('unique_dict.json', 'r') as file:
    prediction_encoder = json.load(file)

print(prediction_encoder)
prediction_decoder = {v: k for k, v in prediction_encoder.items()}
print(prediction_decoder)

{'Light Rain with Thunder': 0, 'Clear and Sunny': 1, 'Partly Cloudy': 2, 'Light Precipitation': 3, 'Cloudy and Overcast': 4, 'Mist or Fog': 5, 'Rain Showers': 6, 'Moderate to Heavy Rain': 7, 'Thunderstorms': 8}
{0: 'Light Rain with Thunder', 1: 'Clear and Sunny', 2: 'Partly Cloudy', 3: 'Light Precipitation', 4: 'Cloudy and Overcast', 5: 'Mist or Fog', 6: 'Rain Showers', 7: 'Moderate to Heavy Rain', 8: 'Thunderstorms'}


In [184]:
predictions = model_retrained.predict(df_input_processed)

In [185]:
predictions[:10]

array([4, 4, 1, 1, 4, 4, 4, 1, 1, 1])

In [186]:
# Decode predictions (assuming you have a dict named 'prediction_decoder' for this purpose)
decoded_predictions = [prediction_decoder[pred] for pred in predictions]

# Add decoded predictions back to df_input
df_input['predictions'] = decoded_predictions
daily_data = pd.read_csv('daily_data.csv')
daily_data['condition_flag'] = daily_data['condition_text'].isna().astype(int)

temp_df = df_input[['city_id', 'day_id', 'predictions']].copy()

result = pd.merge(daily_data, temp_df, on=['city_id', 'day_id'], how='left')

result['condition_text'] = result['condition_text'].fillna(result['predictions'])

result.drop('predictions', axis=1, inplace=True)

In [187]:
result.to_csv('predictions_with_data.csv', index=False)

In [188]:
result[["day_id", "condition_text"]].to_csv('submit.csv', index=False)

In [189]:
print(result["condition_text"].value_counts())

condition_text
Clear and Sunny            1154
Cloudy and Overcast         742
Partly Cloudy               566
Light Precipitation         218
Mist or Fog                 137
Light Rain with Thunder      22
Rain Showers                 22
Thunderstorms                19
Moderate to Heavy Rain       13
Name: count, dtype: int64
